In [ ]:
!pip install TeleBot
!pip install pyTelegramBotAPI
!pip install telegram
!pip install pydub

In [2]:
import os
import telebot
import telegram
import tensorflow as tf
import numpy as np
import pydub

In [20]:
bot = telebot.TeleBot("6370453544:AAEI0zlqAp9HaPkuQv3o448Qs5YoYN1qSbs")
model = tf.keras.models.load_model("audio_classification_model.h5")

In [21]:
@bot.message_handler(commands=["start"])
def send_welcome(message):
    bot.send_message(message.chat.id , "Hello " + message.from_user.first_name + "\nWelcome to Audio Recognizer bot🎙🎵🔊"
                     + "\n\nSend a voice and wait for the result ... " )


@bot.message_handler(content_types=['voice'])
def Audio_recognizer(message):
  file_id = message.voice.file_id
  new_file = bot.get_file(file_id)
  file_path = bot.get_file(file_id).file_path
  downloaded_file = bot.download_file(file_path)
  try:
      with open("/content/data/voice.ogg", 'wb') as new_file:
          new_file.write(downloaded_file)
  except Exception as e:
      bot.reply_to(message, "Error saving file.")
      return


  audio = pydub.AudioSegment.from_file("/content/data/voice.ogg")
  audio_array = audio.get_array_of_samples()
  silent_chunks = pydub.silence.split_on_silence(audio , min_silence_len=2000 , silence_thresh=-45)
  result = sum(silent_chunks)
  result.export("/content/wav_data/voice.wav")
  audio = pydub.AudioSegment.from_file("/content/wav_data/voice.wav")
  chunks = pydub.utils.make_chunks(audio , 1000)

  for i , chunk in enumerate(chunks) :
      if len(chunk) >= 1000 :
          chunk.export(os.path.join("/content/dataset/z" , "fvoice.wav") , format="wav")


  test_data = tf.keras.utils.audio_dataset_from_directory(
    "/content/dataset",
    shuffle=False,
    batch_size=2,
    output_sequence_length= 48000,
    ragged= False,
    labels="inferred",
    label_mode="categorical",
    sampling_rate= None,
    seed=59
  )

  classes_labels = np.array(['abdollah', 'azra', 'davood', 'javad', 'khadijeh', 'kiana','maryam', 'matin',
                             'melika', 'mohammad', 'mohammad_parvari','mona', 'nahid', 'nima', 'omid', 'parisa',
                             'parsa', 'sadegh', 'sajedeh', 'shima', 'tara'])
  predictions = model.predict(test_data)
  probability = predictions
  predicted_class_index = np.argmax(probability)
  print(predicted_class_index)
  predicted_class = classes_labels[predicted_class_index]
  print(predicted_class)
  print(f'The owner of this voice is : {predicted_class}')
  bot.send_message(chat_id= message.chat.id , text=f'The owner of this voice is  :  {predicted_class}')



bot.infinity_polling()

AwACAgQAAxkBAAPVZYcpQChdTVE_ZRIemBOK12suTOsAAvkRAALvoDlQszQj1OKE6W8zBA
Found 1 files belonging to 1 classes.
1/1 [==============================] - 0s 214ms/step
19
shima
The owner of this voice is : shima
AwACAgQAAxkBAAPXZYcpRohaxvTq-dNyie0qxDE1Y38AAvoRAALvoDlQ5EagK66LD8IzBA
Found 1 files belonging to 1 classes.
1/1 [==============================] - 0s 32ms/step
5
kiana
The owner of this voice is : kiana
AwACAgQAAxkBAAPZZYcpTtSVYnr6ifgjDHu5LoxC6qcAAvsRAALvoDlQ4zCdke8AAVZgMwQ
Found 1 files belonging to 1 classes.
1/1 [==============================] - 0s 33ms/step
20
tara
The owner of this voice is : tara
AwACAgQAAxkBAAPbZYcpVCGPocg-MpEH9u81TFc1wpEAAvwRAALvoDlQgfV6FBq8RdszBA
Found 1 files belonging to 1 classes.
1/1 [==============================] - 0s 33ms/step
0
abdollah
The owner of this voice is : abdollah
AwACAgQAAxkBAAPdZYcpYNt8p95sUblrf-vXUQM2wLQAAv0RAALvoDlQ0HWSPAnuCpUzBA
Found 1 files belonging to 1 classes.
1/1 [==============================] - 0s 35ms/step
0
abdollah
The 

2023-12-23 18:41:08,001 (__init__.py:968 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
ERROR:TeleBot:Infinity polling: polling exited
2023-12-23 18:41:08,008 (__init__.py:970 MainThread) ERROR - TeleBot: "Break infinity polling"
ERROR:TeleBot:Break infinity polling
